In [3]:
from oddsapi import OddsApiClient
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime as dt
import os

client = OddsApiClient(api_key=key)


In [4]:
url = "https://api.the-odds-api.com"
ext = "/v3/sports/"
params = {"apiKey": key}
r = requests.get(url=url + ext, params=params)
r.status_code

200

In [ ]:
r.content
sports = json.loads(r.text)
sports['data'][0]
for i in sports['data']:
    print(i['title'], i['active'])

In [22]:

url = "https://api.the-odds-api.com"
req = f"/v3/odds/"
params = {
    "apiKey": key,
    "sport" :'baseball_mlb',
"region" :'us',
'mkt':'h2h',
'oddsFormat': 'american' 
}
odds_req = requests.get(url + req, params=params)

In [24]:
odds = json.loads(odds_req.text)
odds = odds['data']
# odds = odds[0]
arb = []
for game in odds:
    if dt.now() > dt.fromtimestamp(game['commence_time']):
        continue
    g = Game(game)
    if g.arb_exists():
        print(f"Arb exists for {g.away} @ {g.home} at {g.start_time}! Secure that bag!")
        arb.append(game)
print(len(odds))
for game in arb:
    print(game.beat_bookies())


15


In [ ]:
class Odds(object):
    '''
    Object to get data from OddsApi request,
    and parse using Game objects.
    '''

    def __init__(self, sport):
        self.__api_key = os.getenv('api_key')
        base_url = "https://api.the-odds-api.com"
        odds_endpoint = f"/v3/odds/"
        params = {
            "apiKey": self.__api_key,
            "sport" :'baseball_mlb',
            "region" :'us',
            'mkt':'h2h',
            'oddsFormat': 'american'
        }
        odds_req = requests.get(url + req, params=params)
        odds = json.loads(odds_req.text)
        odds = odds['data']
        self.games = []
        for game in odds:
            g = Game(game)
            self.games.append(g)
        

    
class OddsLogger(Odds):
    """
    Object to automatically odds for backtesting purposes. 

    Args:
        sport (str): the sport to log the odds on the sportsbooks. Development to come
    """

In [17]:
from fractions import Fraction
from sympy import symbols, Eq, solve
MY_SPORTSBOOKS = ['Barstool Sportsbook', 'DraftKings', "BetMGM", 'BetRivers', 'FanDuel', 'PointsBet (US)', 'William Hill (Ceasars)']

class Game(object):
    '''
    Object used to organize data from OddsAPI client 
    for processing later
    Inputs:
        data - (dict) data from OddsAPI response. 
        Must include: 'sports_key', 'teams', 'home_team', 
                    'sites'

    '''
    def __init__(self, data):
        self.sport = data['sport_nice']
        self.teams = data['teams']
        self.home = data['home_team']
        for team in self.teams:
            if team != self.home:
                self.away = team
        self.h2h_odds = {}
        for site in data['sites']:
            site_name = site['site_nice']
            self.h2h_odds[site_name] = site['odds']['h2h']
        # self.h2h_odds = pd.DataFrame(self.h2h_odds).T
        # self.h2h_odds.columns = self.teams
        self.start_time = dt.fromtimestamp(int(data['commence_time']))
        self.num_sites = ['sites_count']
        self.best_bettable_odds()
    
    def __repr__(self):
        return f"Game object for {self.away} @ {self.home} on {self.start_time}"

    def best_bettable_odds(self):
        '''
        Iterates through the odds and finds the best odds on both sides 
        Currently only supports games with no draws
        '''
        best_team1 = None
        best_team1_site = None
        best_team2 = None
        best_team2_site = None
        for site, odds in self.h2h_odds.items():
            if site in MY_SPORTSBOOKS:
                if not best_team1:
                    best_team1 = odds[0]
                    best_team1_site = site
                    best_team2 = odds[1]
                    best_team2_site = site
                else:
                    if odds[0] > best_team1:
                        best_team1 = odds[0]
                        best_team1_site = site
                    if odds[1] > best_team2:
                        best_team2 = odds[1]
                        best_team2_site = site
                self.best_odds = {
                    f"{self.teams[0]}": (best_team1, best_team1_site),
                    f"{self.teams[1]}": (best_team2, best_team2_site)
                }
        # print(self.best_odds)

    def arb_exists(self):
        sure_bet = 0
        for odds in self.best_odds.values():
            d_odds = Game.decimal_odds(odds[0])
            sure_bet += 1 / d_odds
        if sure_bet < 1:
            return True
        else:
            return False

    @staticmethod
    def decimal_odds(odds: int) -> float:
        """
        :param odds: Integer (e.g., -350) or String (e.g., '3/1' or '5/4').
        :return: Float. Odds expressed in Decimal terms.
        """
        if isinstance(odds, float):
            return odds

        elif isinstance(odds, int):
            if odds >= 100:
                return abs(1 + (odds / 100))
            elif odds <= -101 :
                return 100 / abs(odds) + 1
            else:
                return float(odds)

        elif "/" in odds:
            odds = Fraction(odds)
            return round((odds.numerator / odds.denominator) + 1, 2)
        
    def beat_bookies(self, total_stake):
        odds1 = self.best_odds[self.teams[0]][0]
        odds2 = self.best_odds[self.teams[1]][0]
        x, y = symbols('x y')
        eq1 = Eq(x + y - total_stake, 0) # total_stake = x + y
        eq2 = Eq((odds2*y) - odds1*x, 0) # odds1*x = odds2*y
        stakes = solve((eq1,eq2), (x, y))
        total_investment = stakes[x] + stakes[y]
        profit1 = odds1*stakes[x] - total_stake
        profit2 = odds2*stakes[y] - total_stake
        print(type(profit1), type(total_investment))
        benefit1 = f'{round(profit1 / total_investment * 100,2)}%'
        benefit2 = f'{round(profit2 / total_investment * 100,2)}%'
        dict_gabmling = {f'{self.teams[0]} stake':stakes[x], f'{self.teams[1]} stake':stakes[y], 'Profit1':profit1, 'Profit2':profit2,
                        'Benefit1': benefit1, 'Benefit2': benefit2}
        return dict_gabmling

        

g = Game(odds)
g.beat_bookies(500)

<class 'sympy.core.numbers.Integer'> <class 'sympy.core.numbers.Integer'>


{'Detroit Tigers stake': 29500,
 'New York Yankees stake': -29000,
 'Profit1': 8554500,
 'Profit2': 8554500,
 'Benefit1': '1710900%',
 'Benefit2': '1710900%'}

In [9]:
g.h2h_odds

{'BetOnline.ag': [241, -270],
 'MyBookie.ag': [245, -305],
 'FOX Bet': [255, -357],
 'DraftKings': [250, -350],
 'WynnBET': [250, -278],
 'GTbets': [267, -363],
 'BetUS': [230, -280],
 'William Hill (US)': [240, -280],
 'Bovada': [260, -375],
 'LowVig.ag': [241, -270],
 'PointsBet (US)': [285, -375],
 'FanDuel': [290, -410],
 'BetMGM': [280, -400],
 'Unibet': [280, -385],
 'TwinSpires': [280, -385],
 'BetRivers': [245, -295],
 'SugarHouse': [280, -385],
 'Barstool Sportsbook': [280, -385],
 'Intertops': [240, -280],
 'Betfair': [168, -588]}

In [13]:
# g.best_bettable_odds()
# g.arb_exists()
# g.beat_bookies(50)

TypeError: unsupported format string passed to Integer.__format__